In [3]:
import os
from paths import *
import statistics
import itertools
import numpy as np
import random
import pandas as pd

In [4]:
def get_directory_size(directory):
    total_size = 0
    for path, dirs, files in os.walk(directory):
        for f in files:
            fp = os.path.join(path, f)
            total_size += os.path.getsize(fp)
    return total_size

# Example usage
directory_path = wav_path
size = get_directory_size(directory_path)
print(f"The total size of the directory '{directory_path}' is {size/(1024*1024*1024): .3f} GB.")

The total size of the directory '../src/bsc/wav/' is  4.074 GB.


In [5]:
ssd_base_path = "/media/ldlmdl/A2AAE4B1AAE482E1/SSD_Documents/featln/"
ssd_src_path = ssd_base_path + "src/"
ssd_aishell_path = ssd_src_path + "aishell/"
as_original_path = ssd_aishell_path + "original/"

In [ ]:
def get_directory_size(directory):
    total_size = 0
    for path, dirs, files in os.walk(directory):
        for f in files:
            fp = os.path.join(path, f)
            total_size += os.path.getsize(fp)
    return total_size

def select_subdirs(directory, target_size):
    subdirs = [subdir for subdir in os.listdir(directory) if os.path.isdir(os.path.join(directory, subdir))]
    subdirs_sizes = [(subdir, get_directory_size(os.path.join(directory, subdir))) for subdir in subdirs]
    subdirs_sizes.sort(key=lambda x: x[1])  # Sort subdirectories based on total size

    selected_subdirs = []
    selected_size = 0
    selected_sizes = []
    for subdir, size in subdirs_sizes:
        if selected_size + size <= target_size:
            selected_subdirs.append((subdir, size))
            selected_size += size
            selected_sizes.append(size)

    # Calculate the variance of selected sizes
    variance = statistics.variance(selected_sizes)
    print(variance)

    # If variance is greater than a threshold, remove the last selected subdirectory
    threshold = 0.1  # Adjust the threshold as per your requirement
    if variance > threshold and len(selected_subdirs) > 1:
        selected_subdirs.pop()
        selected_size -= selected_sizes[-1]

    return selected_subdirs

# Example usage
directory_path = as_original_path
target_size = 4.074 * (pow(1024, 3))  # Target size in GB
selected_subdirs = select_subdirs(directory_path, target_size)
print("Selected subdirectories:")
for subdir, size in selected_subdirs:
    print(f"Subdirectory: {subdir}")
    print(f"Total size: {size} bytes")


In [6]:
def get_directory_size(directory):
    total_size = 0
    for path, dirs, files in os.walk(directory):
        for f in files:
            fp = os.path.join(path, f)
            total_size += os.path.getsize(fp)
    return total_size

In [7]:
def find_sublist(arr, target_sum, max_variance):
    best_sublist = []
    best_variance = float('inf')

    for r in range(1, len(arr) + 1):
        for combo in itertools.combinations(arr, r):
            current_sum = sum(combo)
            current_variance = np.var(combo)

            if abs(current_sum - target_sum) < abs(sum(best_sublist) - target_sum) and current_variance <= max_variance:
                best_sublist = list(combo)
                best_variance = current_variance

    return best_sublist

In [8]:
subdirs = [subdir for subdir in os.listdir(as_original_path) if os.path.isdir(os.path.join(as_original_path, subdir))]
subdirs_sizes = [(subdir, get_directory_size(os.path.join(as_original_path, subdir))) for subdir in subdirs]
subdirs_sizes.sort(key=lambda x: x[1])  # Sort subdirectories based on total size
names_list, sizes_list = zip(*subdirs_sizes)

In [52]:
import os

# Get the list of filenames in the directory
filenames = sorted(os.listdir(wav_path))

# Create a dictionary to store the speakers
speakers = {}

# Categorize the filenames into speakers
for filename in filenames:
    speaker = filename[:3]
    if speaker not in speakers:
        speakers[speaker] = []
    speakers[speaker].append(filename)


In [ ]:
sizes_ = []
for speaker, files in speakers.items():
    print(f"Speaker: {speaker}")
    print(f"Number of files: {len(files)}")
    total_size = 0
    for file in files:
        total_size += os.path.getsize(os.path.join(wav_path, file))

    total_size /= 1048576
    sizes_.append(total_size)
    print(f"Total size: {total_size:.3f} MB")

In [9]:
sizes_ = np.array(sizes_)
sizes_.mean()

NameError: name 'sizes_' is not defined

In [10]:
rnl = sorted(names_list, reverse=True)

In [11]:
resample_ratio = 2.7546

In [13]:
def select_files(directory, target_size):
    files = os.listdir(directory)
    random.shuffle(files)  # Shuffle the list of files randomly

    selected_files = []
    selected_size = 0

    for file in files:
        file_path = os.path.join(directory, file)
        file_size = os.path.getsize(file_path)

        if selected_size + file_size <= target_size:
            selected_files.append(file)
            selected_size += file_size

    return selected_files

In [ ]:
df = pd.DataFrame(columns=['speaker', 'rec'])
for subdir in rnl: 
    # Example usage
    directory_path = os.path.join(as_original_path, subdir)
    target_size = 104.28599400520325 * (1024*1024) * resample_ratio # Target size in bytes (e.g., 1 GB)
    selected_files = select_files(directory_path, target_size)
    selected_files = sorted(selected_files)
    add_content = pd.DataFrame({'speaker': [subdir] * len(selected_files), 'rec': selected_files})
    df = pd.concat([df, add_content], axis=0)

In [15]:
df.to_csv(ssd_aishell_path + 'filelog.csv', index=False)

In [68]:
import torchaudio

In [69]:
tp = os.path.join(as_original_path, "SSB0005", "SSB00050024.wav")

In [70]:
wave, sr = torchaudio.load(tp)

In [74]:
rswave = torchaudio.functional.resample(wave, sr, 16000)

In [76]:
rswave.shape, wave.shape

(torch.Size([1, 56112]), torch.Size([1, 154657]))

In [78]:
torchaudio.save("test.wav", rswave, 16000)

In [79]:
os.path.getsize(tp) / os.path.getsize("test.wav")

2.7546971558832434